In [2]:
import pandas as pd
import json
import traceback

import sys
sys.path.insert(1, '../')
import modules as md
ROOT_DIR = "../"

In [12]:
statement = pd\
    .read_csv(ROOT_DIR + md.BANK_STATEMENTS_DIR + 'blue_card.csv',header=0)\
    .rename(columns={
        'Date': 'date',
        'Description':'description',
        'Debits and credits':'amount'
    })[['date','description','amount']]

statement.iloc[0].to_dict()

{'date': '30/08/2023',
 'description': 'Transfer to Lachlan D Shewring BSB 014111 Account 436477678 Air BNB Andre',
 'amount': ' -$150.00'}

# creating class

In [ ]:
class Statements:
    def __init__(self, accounts = md.STATEMENT_ACCOUNTS):

        self.statments = {}
        self.accounts = md.STATEMENT_ACCOUNTS
    
    def load_statment(self, account):
        return pd\
            .read_csv(ROOT_DIR + md.BANK_STATEMENTS_DIR + '/' + account + '.csv', header=0)\
            .rename(columns={
                'Date': 'date',
                'Description':'description',
                'Debits and credits':'amount'
            })[['date','description','amount']]
    

    def get_transaction(self, account, transaction):
        
        if account not in self.accounts:
            return "sorry, could not find this account"

        if not(account in self.statments.keys()):
            try:
                self.statments[account] = self.load_statment(account)
            except Exception as e:
                traceback.print_exc()
                return "sorry, error loading statement"
            
        if transaction > self.statments[account].shape[0]:
            return "no more transactions!"
        
        return self.statments[account].iloc[transaction].to_dict()


statements = Statements()



# basic predicting transaction

In [7]:
history = pd\
    .read_csv(ROOT_DIR + md.HISTORIC_CLASSIFICATIONS,header=0)\
    # .rename(columns={
    #     'Date': 'date',
    #     'Description':'description',
    #     'Debits and credits':'amount'
    # })[['date','description','amount']]

history

,movement,date,where,description_full,code,description_short
0,input,28-Aug-2023,pink_card,Purchase MCLI PTY LTD MELBOURNE,0=W,no clue
1,input,28-Aug-2023,pink_card,Purchase MCLI PTY LTD MELBOURNE,0=W,no clue
2,input,28-Aug-2023,pink_card,Purchase MCLI PTY LTD MELBOURNE,0=W,no clue
3,input,28-Aug-2023,pink_card,Purchase MCLI PTY LTD MELBOURNE,0=W,no clue
4,input,13-Aug-2023,blue_card,Transfer from Andre Medina BSB 944600 Account ...,3=GS,no clue
5,input,13-Aug-2023,be,Transfer from Andre Medina BSB 944600 Account ...,3=GS,no clue
6,input,27-Aug-2023,bd,Purchase BUZA CHICKEN - MELBOUR MELBOURNE VIC,3=GS,no clue
7,output,27-Aug-2023,bd,Purchase BUZA CHICKEN - MELBOUR MELBOURNE VIC,3=GS,no clue
8,output,27-Aug-2023,bd,Purchase BUZA CHICKEN - MELBOUR MELBOURNE VIC,6=FEh,chicken with boys
9,output,30-Aug-2023,be,Transfer to Lachlan D Shewring BSB 014111 Acco...,1=fe,amedina3


In [64]:

selected_history = history[history['description_full'] == statement.iloc[0].to_dict()['description']]

code = selected_history['code'].mode()[0]
current_movement_type = selected_history['movement'].mode()[0]
short_description = selected_history['description_short'].mode()[0]

code_half = code.split('=')[1].split('-')
selected_levels = [code[0]] + [*(code_half[0])]


description_tag = {'tag': code_half[1][0],'name': code_half[1][1:]} if len(code_half) > 1 else {'tag':None,'name': None}

prediction = {
    'description_tag':description_tag,
    'selected_levels':selected_levels,
    'short_description':short_description,
    'current_movement_type':current_movement_type
}

prediction

{'description_tag': {'tag': None, 'name': None},
 'selected_levels': ['1', 'f', 'e'],
 'short_description': 'amedina3',
 'current_movement_type': 'input'}

In [56]:
code = history[history['description_full'] == statement.iloc[0].to_dict()['description']]['code'].mode()[0]
# code += 'asfd-lmarcus'



selected_levels

['1', 'f', 'e']